In [ ]:
import os, sys
currentdir = os.path.dirname(os.path.realpath("__file__"))
parentdir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath("__file__"))))
sys.path.append(parentdir)

from joblib import Parallel, delayed
import numpy as np
import pickle 
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torchmetrics
import seaborn as sns
import glob
from tqdm import tqdm
from copy import deepcopy
from src.transformer.metrics import AUL, CorrectedMCC, CorrectedBAcc, CorrectedF1
from src.analysis.metrics.cartesian import cartesian_jit

from torchmetrics import MatthewsCorrCoef, F1Score, Accuracy
from sklearn.utils import resample
from scipy.stats import median_abs_deviation as mad
import time
import cmcrameri.cm as cmc
import matplotlib.colors as clr
from matplotlib.ticker import MultipleLocator, LinearLocator, AutoMinorLocator
plt.rcParams["grid.linestyle"] =  ":"
plt.rcParams["axes.edgecolor"] = "gray"
plt.rcParams["axes.linewidth"] = 0.7

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Helvetica"
#sns.set_context("notebook", font_scale=1.2)
#import scienceplots
#plt.style.use(["science", "nature"])

In [ ]:
cmap = cmc.batlowS
cmap

In [ ]:
save_path = r".../analysis/plots/hexaco/"

In [ ]:
mean = {"rnd": [.169, .177, .185, .174],
        "rnn": [.111, .030, .050, .010],
        "l2v": [.223, .237, .232, .220],
        }
ci_l = {"rnd":  [.151, .157, .166, .156],
        "rnn":  [.111, .030, .048, .006],
        "l2v":  [.198, .217, .210, .200]}

ci_u = {"rnd":  [.188, .198, .205, .193],
        "rnn":  [.121, .040, .059, .012],
        "l2v":  [.251, .258, .255, .240]}

fig, ax = plt.subplots(1,2, figsize=(15,5))
x_tick = np.array([1,2,3,4])
for i, k in enumerate(mean.keys()):
    yerr = np.vstack([np.array(np.abs(np.array(mean[k]) - np.array(ci_l[k]))),
                     np.array(np.abs(np.array(mean[k]) - np.array(ci_u[k])))])
    ax[0].bar(x = x_tick + i*0.2, height = mean[k], yerr=yerr, width=0.2, capsize=5)
ax[0].set_xticks(x_tick + 0.2)
ax[0].set_xticklabels( ["Q1", "Q2", "Q3", "Q4"])
ax[0].set_title("Macro F1-Score per Personality Aspect")
ax[0].set_ylabel("Score")
ax[0].set_xlabel("Aspect")
ax[0].tick_params(axis= "y", which="major", width=1, length = 6, direction="out", color="gray")
ax[0].tick_params(axis= "y", which="minor", width=1, length =3, direction="out", color="gray")
ax[0].tick_params(axis= "x", which="both", width=1 , length =1)
ax[0].yaxis.set_major_locator(MultipleLocator(0.05))
ax[0].yaxis.set_minor_locator(AutoMinorLocator(5))
sns.despine()

mean = {"rnd": [-0.025, -0.013, 0.009, 0.010],
        "rnn": [ 0.106,  0.086, 0.143, 0.096],
        "l2v": [ 0.126,  0.129, 0.229, 0.109],
        }

mad = {"rnd":  [ 0.013,  0.018, 0.017, 0.013],
        "rnn": [ 0.017,  0.017, 0.017, 0.017],
        "l2v": [ 0.018,  0.015, 0.017, 0.018],
        }



x_tick = np.array([1,2,3,4])
for i, k in enumerate(mean.keys()):
    yerr = np.array(mad[k])

    ax[1].bar(x = x_tick + i*0.2, height = mean[k], yerr=yerr, width=0.2, capsize=5)
ax[1].set_xticks(x_tick + 0.2)
ax[1].set_xticklabels( ["Q1", "Q2", "Q3", "Q4"])
ax[1].set_title("Cohen's Quadratic Kappa Score per Personality Aspect")
ax[1].set_ylabel("Score")
ax[1].set_xlabel("Aspect")
ax[1].tick_params(axis= "y", which="major", width=1, length = 6, direction="out", color="gray")
ax[1].tick_params(axis= "y", which="minor", width=1, length =3, direction="out", color="gray")
ax[1].tick_params(axis= "x", which="both", width=1 , length =1)
ax[1].yaxis.set_major_locator(MultipleLocator(0.05))
ax[1].yaxis.set_minor_locator(AutoMinorLocator(5))
ax[1].axhline(0.0, color="gray", linewidth=0.5, linestyle= ":")

sns.despine()

plt.savefig(save_path + "performance.svg", format="svg")

plt.show()

In [ ]:
means = {"l2v": {"HEXACO_extra9R": .091,
                 "HEXACO_extra8":  .068, 
                 "HEXACO_extra5":  .107, 
                 "HEXACO_extra10R":.146, 
                 "HEXACO_extra7":  .013, 
                 "HEXACO_extra2R": .121, 
                 "HEXACO_extra6":  .163, 
                 "HEXACO_extra4":  .061, 
                 "HEXACO_extra3R": .134, 
                 "HEXACO_extra1":  .061,
                 }, 
         "rnn": {"HEXACO_extra9R": .074,
                 "HEXACO_extra8":  .052, 
                 "HEXACO_extra5":  .097, 
                 "HEXACO_extra10R":.088, 
                 "HEXACO_extra7":  .046, 
                 "HEXACO_extra2R": .019, 
                 "HEXACO_extra6":  .131, 
                 "HEXACO_extra4":  .069, 
                 "HEXACO_extra3R": .060, 
                 "HEXACO_extra1":  .046,
                 },
         "rnd": {"HEXACO_extra9R": -.012,
                 "HEXACO_extra8":  -.007, 
                 "HEXACO_extra5":  .015, 
                 "HEXACO_extra10R":-.008, 
                 "HEXACO_extra7":  .022, 
                 "HEXACO_extra2R": .006, 
                 "HEXACO_extra6":  -.012, 
                 "HEXACO_extra4":  .002, 
                 "HEXACO_extra3R": -.007, 
                 "HEXACO_extra1":  -.008,
         }}

ses =  {"l2v": {"HEXACO_extra9R": .019,
                 "HEXACO_extra8": .019, 
                 "HEXACO_extra5": .018, 
                 "HEXACO_extra10R":.020, 
                 "HEXACO_extra7": .018, 
                 "HEXACO_extra2R": .018, 
                 "HEXACO_extra6": .019, 
                 "HEXACO_extra4": .018, 
                 "HEXACO_extra3R": .018, 
                 "HEXACO_extra1": .019,
                 }, 
         "rnn": {"HEXACO_extra9R": .019,
                 "HEXACO_extra8": .015, 
                 "HEXACO_extra5": .016, 
                 "HEXACO_extra10R": .019, 
                 "HEXACO_extra7": .017, 
                 "HEXACO_extra2R": .009, 
                 "HEXACO_extra6": .018, 
                 "HEXACO_extra4": .017, 
                 "HEXACO_extra3R": .017, 
                 "HEXACO_extra1": .016,
                 }, 
         "rnd": {"HEXACO_extra9R": .013,
                 "HEXACO_extra8":  .013, 
                 "HEXACO_extra5":  .018, 
                 "HEXACO_extra10R": .017, 
                 "HEXACO_extra7": .018, 
                 "HEXACO_extra2R": .016, 
                 "HEXACO_extra6": .016, 
                 "HEXACO_extra4": .017, 
                 "HEXACO_extra3R": .015, 
                 "HEXACO_extra1":  .014,
         }}

fig, ax = plt.subplots(1,2, figsize=(20,5))
x_tick = np.array([1,2,3,4,])
labels = ["HEXACO_extra9R", "HEXACO_extra8", "HEXACO_extra10R", 
          "HEXACO_extra5", "HEXACO_extra4", "HEXACO_extra3R",  
          "HEXACO_extra7", "HEXACO_extra6",
          "HEXACO_extra2R",   "HEXACO_extra1"]

x_tick = np.array([i for i in range(10)])
for i, k in enumerate(means.keys()):
    yerr = np.array([ses[k][l] for l in labels])
    ax[0].bar(x = x_tick + i*0.3, height = [means[k][l] for l in labels], yerr=yerr, width=0.3, capsize=1.5)
ax[0].set_xticks(x_tick + 0.3)
ax[0].set_xticklabels( labels, rotation=90)
ax[0].set_title("Cohen's Linear Kappa Score per Personality Aspect")
ax[0].set_ylabel("Score")
ax[0].set_xlabel("Aspect")
ax[0].tick_params(axis= "y", which="major", width=1, length = 6, direction="out", color="gray")
ax[0].tick_params(axis= "y", which="minor", width=1, length =3, direction="out", color="gray")
ax[0].tick_params(axis= "x", which="both", width=1 , length =1)
ax[0].yaxis.set_major_locator(MultipleLocator(0.05))
ax[0].yaxis.set_minor_locator(AutoMinorLocator(5))
sns.despine()

means = {"l2v": {"HEXACO_extra9R": .140,
                 "HEXACO_extra8": .118, 
                 "HEXACO_extra5": .164, 
                 "HEXACO_extra10R": .212, 
                 "HEXACO_extra7": .034, 
                 "HEXACO_extra2R": .208, 
                 "HEXACO_extra6": .231, 
                 "HEXACO_extra4": .110, 
                 "HEXACO_extra3R": .208, 
                 "HEXACO_extra1": .091,
                 }, 
         "rnn": {"HEXACO_extra9R": .099,
                 "HEXACO_extra8": .100, 
                 "HEXACO_extra5": .150, 
                 "HEXACO_extra10R": .131, 
                 "HEXACO_extra7": .073, 
                 "HEXACO_extra2R": .040, 
                 "HEXACO_extra6": .182, 
                 "HEXACO_extra4": .084, 
                 "HEXACO_extra3R": .091, 
                 "HEXACO_extra1": .057,
                 },
         "rnd": {"HEXACO_extra9R": -.018,
                 "HEXACO_extra8": -.015, 
                 "HEXACO_extra5": .019, 
                 "HEXACO_extra10R": -.001, 
                 "HEXACO_extra7": .045, 
                 "HEXACO_extra2R": .005, 
                 "HEXACO_extra6": .015, 
                 "HEXACO_extra4": -.006, 
                 "HEXACO_extra3R": -.004, 
                 "HEXACO_extra1": -.014,
                },
         "prm": {"HEXACO_extra9R": -.020,
                 "HEXACO_extra8":   .028, 
                 "HEXACO_extra5":   .011, 
                 "HEXACO_extra10R": .014, 
                 "HEXACO_extra7":  -.015, 
                 "HEXACO_extra2R": -.027, 
                 "HEXACO_extra6":   .025, 
                 "HEXACO_extra4":  -.006, 
                 "HEXACO_extra3R":  .017, 
                 "HEXACO_extra1":  -.003,
                }}

ses =  {"l2v": {"HEXACO_extra9R": .027,
                 "HEXACO_extra8": .029, 
                 "HEXACO_extra5": .026, 
                 "HEXACO_extra10R": .027, 
                 "HEXACO_extra7": .026, 
                 "HEXACO_extra2R": .026, 
                 "HEXACO_extra6": .026, 
                 "HEXACO_extra4": .026, 
                 "HEXACO_extra3R": .026, 
                 "HEXACO_extra1": .028,
                 }, 
         "rnn": {"HEXACO_extra9R": .027,
                 "HEXACO_extra8": .023, 
                 "HEXACO_extra5": .023, 
                 "HEXACO_extra10R": .026, 
                 "HEXACO_extra7": .025, 
                 "HEXACO_extra2R": .019, 
                 "HEXACO_extra6": .025, 
                 "HEXACO_extra4": .025, 
                 "HEXACO_extra3R": .025, 
                 "HEXACO_extra1": .026,
                 }, 
         "rnd": {"HEXACO_extra9R": .016,
                 "HEXACO_extra8":  .019, 
                 "HEXACO_extra5":  .026, 
                 "HEXACO_extra10R": .024, 
                 "HEXACO_extra7": .025, 
                 "HEXACO_extra2R": .023, 
                 "HEXACO_extra6": .025, 
                 "HEXACO_extra4": .025, 
                 "HEXACO_extra3R": .022, 
                 "HEXACO_extra1":  .018,
                },
         "prm": {"HEXACO_extra9R":  .026,
                 "HEXACO_extra8":   .027, 
                 "HEXACO_extra5":   .027, 
                 "HEXACO_extra10R": .027, 
                 "HEXACO_extra7":   .026, 
                 "HEXACO_extra2R":  .026, 
                 "HEXACO_extra6":   .027, 
                 "HEXACO_extra4":   .026, 
                 "HEXACO_extra3R":  .027, 
                 "HEXACO_extra1":   .027,
                }}

labels = ["HEXACO_extra9R", "HEXACO_extra8", "HEXACO_extra10R", 
          "HEXACO_extra5", "HEXACO_extra4", "HEXACO_extra3R",  
          "HEXACO_extra7", "HEXACO_extra6",
          "HEXACO_extra2R",   "HEXACO_extra1"]

del means["rnd"]
del ses["rnd"]

x_tick = np.array([i for i in range(10)])
for i, k in enumerate(means.keys()):
    
    yerr = np.array([ses[k][l] for l in labels])
    #yerr = np.vstack([np.array(np.abs(np.array(mean[k]) - np.array(ci_l[k]))),
    #                 np.array(np.abs(np.array(mean[k]) - np.array(ci_u[k])))])
    ax[1].bar(x = x_tick + i*0.3, height = [means[k][l] for l in labels], yerr=yerr, width=0.3, capsize=1.5)
ax[1].set_xticks(x_tick + 0.3)
ax[1].set_xticklabels( labels, rotation=90)
ax[1].set_title("Cohen's Quadratic Kappa Score per Personality Aspect")
ax[1].set_ylabel("Score")
ax[1].set_xlabel("Aspect")
ax[1].tick_params(axis= "y", which="major", width=1, length = 6, direction="out", color="gray")
ax[1].tick_params(axis= "y", which="minor", width=1, length =3, direction="out", color="gray")
ax[1].tick_params(axis= "x", which="both", width=1 , length =1)
ax[1].yaxis.set_major_locator(MultipleLocator(0.05))
ax[1].yaxis.set_minor_locator(AutoMinorLocator(5))
ax[1].axhline(0.0, color="gray", linewidth=0.5, linestyle= ":")

ax[1].axvline(2.8, color="gray", linewidth=0.5, linestyle= ":")
ax[1].axvline(5.8, color="gray", linewidth=0.5, linestyle= ":")
ax[1].axvline(7.8, color="gray", linewidth=0.5, linestyle= ":")



sns.despine()

plt.savefig(save_path + "performance2.svg", format="svg")

plt.show()